## Importing all the Nessary Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import json

warnings.filterwarnings('ignore')

# Initialize global variables
data_dir = Path('./processed')
results = {}

## Data Cleaning

In [2]:
# loading all the datasets to be used
# Load ILOSTAT labour data
print("\n[1/4] Loading ILOSTAT Labour Data...")
ilostat_df = pd.read_csv('data/combined_ilostat_append_africa.csv' )
print(f"   ✓ Loaded {len(ilostat_df)} records")

# Load SDG National data
print("\n[2/4] Loading SDG National Education Data...")
sdg_national_df = pd.read_csv('data/SDG_DATA_NATIONAL_ENRICHED_AFRICA.csv')
print(f"   ✓ Loaded {len(sdg_national_df)} records")

# Load SDG Regional data
print("\n[3/4] Loading SDG Regional Data...")
sdg_regional_df = pd.read_csv('data/SDG_DATA_REGIONAL_ENRICHED_AFRICA.csv')
print(f"   ✓ Loaded {len(sdg_regional_df)} records")

# Load World Bank data
print("\n[4/4] Loading World Bank Data...")
worldbank_df = pd.read_csv('data/world_bank_data_africa.csv')
print(f"   ✓ Loaded {len(worldbank_df)} records")


[1/4] Loading ILOSTAT Labour Data...
   ✓ Loaded 826130 records

[2/4] Loading SDG National Education Data...
   ✓ Loaded 299039 records

[3/4] Loading SDG Regional Data...
   ✓ Loaded 6731938 records

[4/4] Loading World Bank Data...
   ✓ Loaded 1350 records


#### Cleaning ILOstat data

In [3]:
# quick check up and summary of the ilostat data
ilostat_df_copy = ilostat_df.copy()
initial_rows = len(ilostat_df_copy)
#checking available country distributions
print(ilostat_df_copy['country_name'].value_counts())
print(initial_rows)
ilostat_df_copy.head()

country_name
South Africa                        81576
Mauritius                           65322
Egypt                               51882
Mali                                30195
Ghana                               28970
Tanzania                            28944
Rwanda                              27907
Tunisia                             26414
Botswana                            26197
Zambia                              25531
Senegal                             24472
Zimbabwe                            22794
Uganda                              22499
Burkina Faso                        20417
Morocco                             19750
Namibia                             19727
Côte d'Ivoire                       18279
Kenya                               17650
Nigeria                             16951
Seychelles                          14664
Niger                               14456
Malawi                              13209
Ethiopia                            12818
Togo                 

,ref_area,source,indicator,sex,classif1,classif2,time,obs_value,obs_status,note_classif,note_indicator,note_source,source_file,country_name,indicator_label
0,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_TOTAL,2021,13210.662,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
1,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_LTB,2021,2934.573,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
2,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_BAS,2021,6264.182,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
3,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_INT,2021,2137.752,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
4,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_ADV,2021,438.449,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."


In [4]:
# Standardize column names
ilostat_df_copy = ilostat_df_copy.rename(columns={
    'country_name': 'country',
    'time': 'year',
    'obs_value': 'value',
    'ref_area': 'country_id',
    'indicator': '*indicator_id',
    'indicator_label': 'indicator'
})

#checking the standardized column name
ilostat_df_copy.head()

,country_id,source,*indicator_id,sex,classif1,classif2,year,value,obs_status,note_classif,note_indicator,note_source,source_file,country,indicator
0,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_TOTAL,2021,13210.662,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
1,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_LTB,2021,2934.573,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
2,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_BAS,2021,6264.182,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
3,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_INT,2021,2137.752,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
4,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_ADV,2021,438.449,NaN,NaN,NaN,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."


In [5]:
# checking out the shape and missing values of the data
print(ilostat_df_copy.shape)
print(ilostat_df_copy.isnull().sum())
# checking the percentage of missing values
print(ilostat_df_copy.isnull().sum() / len(ilostat_df_copy) * 100)

(826130, 15)
country_id             0
source                 0
*indicator_id          0
sex                  330
classif1            1152
classif2           56330
year                   0
value              83570
obs_status        545044
note_classif      818432
note_indicator    640380
note_source        16475
source_file            0
country                0
indicator              0
dtype: int64
country_id         0.000000
source             0.000000
*indicator_id      0.000000
sex                0.039945
classif1           0.139445
classif2           6.818539
year               0.000000
value             10.115841
obs_status        65.975573
note_classif      99.068185
note_indicator    77.515645
note_source        1.994238
source_file        0.000000
country            0.000000
indicator          0.000000
dtype: float64


In [6]:
# drop all columns with missing percentages greater than 50%
ilostat_df_copy = ilostat_df_copy.dropna(thresh=len(ilostat_df_copy) * 0.5, axis=1)
# checking the shape of the data
print(ilostat_df_copy.shape)
# check the updated data
ilostat_df_copy.head()

(826130, 12)


,country_id,source,*indicator_id,sex,classif1,classif2,year,value,note_source,source_file,country,indicator
0,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_TOTAL,2021,13210.662,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
1,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_LTB,2021,2934.573,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
2,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_BAS,2021,6264.182,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
3,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_INT,2021,2137.752,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."
4,AGO,BA:13951,EAP_TEAP_SEX_AGE_EDU_NB,SEX_T,AGE_YTHADULT_YGE15,EDU_AGGREGATE_ADV,2021,438.449,R1:3513,EAP_TEAP_SEX_AGE_EDU_NB_A-20251129T1913.csv,Angola,"Economically active population by sex, age and..."


In [7]:
# cleaning the rest of the missing values by dropping rows using value as a subset
ilostat_df_copy = ilostat_df_copy.dropna(subset=['value'])
# dropping the rest of the missing rows since they won't affect our data that much
ilostat_df_copy = ilostat_df_copy.dropna()
# check the sum of missing values
print(ilostat_df_copy.isnull().sum())
# check the data's new shape
print(ilostat_df_copy.shape)

country_id       0
source           0
*indicator_id    0
sex              0
classif1         0
classif2         0
year             0
value            0
note_source      0
source_file      0
country          0
indicator        0
dtype: int64
(674384, 12)


In [8]:
# Convert year to integer
ilostat_df_copy['year'] = pd.to_numeric(ilostat_df_copy['year'], errors='coerce')
ilostat_df_copy = ilostat_df_copy.dropna(subset=['year'])
ilostat_df_copy['year'] = ilostat_df_copy['year'].astype(int)

# Convert value to numeric
ilostat_df_copy['value'] = pd.to_numeric(ilostat_df_copy['value'], errors='coerce')

In [9]:
# Standardize country names
ilostat_df_copy['country'] = ilostat_df_copy['country'].str.strip()

# Create indicator category
ilostat_df_copy['indicator_category'] = 'Labour'
ilostat_df_copy['data_source'] = 'ILOSTAT'

In [10]:
# Report of the ILOStat Data Cleaning
print(f"\n✓ Cleaned ILOSTAT data: {initial_rows} → {len(ilostat_df_copy)} rows")
print(f"   • Countries: {ilostat_df_copy['country'].nunique()}")
print(f"   • Year range: {ilostat_df_copy['year'].min()} - {ilostat_df_copy['year'].max()}")
print(f"   • Indicators: {ilostat_df_copy['indicator'].nunique()}")


✓ Cleaned ILOSTAT data: 826130 → 674384 rows
   • Countries: 51
   • Year range: 1982 - 2024
   • Indicators: 5


### Cleaning up SDG Data - National

In [11]:
# quick check up and summary of the ilostat data
sdg_national_df_copy = sdg_national_df.copy()
sdg_rows = len(sdg_national_df_copy)
print(sdg_rows)
sdg_national_df_copy.head()

299039


,INDICATOR_ID,COUNTRY_ID,YEAR,VALUE,MAGNITUDE,QUALIFIER,INDICATOR_LABEL_EN,COUNTRY_NAME_EN
0,ADMI.ENDOFLOWERSEC.MAT,AGO,2014,0.0,NaN,NaN,Administration of a nationally-representative...,Angola
1,ADMI.ENDOFLOWERSEC.MAT,AGO,2015,0.0,NaN,NaN,Administration of a nationally-representative...,Angola
2,ADMI.ENDOFLOWERSEC.MAT,AGO,2016,0.0,NaN,NaN,Administration of a nationally-representative...,Angola
3,ADMI.ENDOFLOWERSEC.MAT,AGO,2017,0.0,NaN,NaN,Administration of a nationally-representative...,Angola
4,ADMI.ENDOFLOWERSEC.MAT,AGO,2018,0.0,NaN,NaN,Administration of a nationally-representative...,Angola


In [12]:
# standardizing the column names
sdg_national_df_copy = sdg_national_df_copy.rename(columns={
    'COUNTRY_NAME_EN': 'country',
    'YEAR': 'year',
    'VALUE': 'value',
    'INDICATOR_LABEL_EN': 'indicator',
    'INDICATOR_ID': 'indicator_id',
    'COUNTRY_ID': 'country_id',
    'MAGNITUDE': 'magnitude',
    'QUALIFIER': 'qualifier',
})

In [13]:
# converting columns to the appropriate data types
sdg_national_df_copy['year'] = pd.to_numeric(sdg_national_df_copy['year'], errors='coerce')
sdg_national_df_copy['value'] = pd.to_numeric(sdg_national_df_copy['value'], errors='coerce')
sdg_national_df_copy['year'] = sdg_national_df_copy['year'].astype(int)
sdg_national_df_copy['country'] = sdg_national_df_copy['country'].str.strip()
sdg_national_df_copy['indicator_category'] = 'Education'
sdg_national_df_copy['data_source'] = 'SDG_National'


In [14]:
# checking out the shape and missing values of the data
print(sdg_national_df_copy.shape)
print(sdg_national_df_copy.isnull().sum())
# checking the percentage of missing values
print(sdg_national_df_copy.isnull().sum() / len(sdg_national_df_copy) * 100)

(299039, 10)
indicator_id               0
country_id                 0
year                       0
value                      0
magnitude             293008
qualifier             263545
indicator                  0
country                    0
indicator_category         0
data_source                0
dtype: int64
indicator_id           0.000000
country_id             0.000000
year                   0.000000
value                  0.000000
magnitude             97.983206
qualifier             88.130645
indicator              0.000000
country                0.000000
indicator_category     0.000000
data_source            0.000000
dtype: float64


In [15]:
# drop columns that have more than 50% missing values
sdg_national_df_copy = sdg_national_df_copy.dropna(thresh=len(sdg_national_df_copy) * 0.5, axis=1)
# checking the shape of the data
print(sdg_national_df_copy.shape)
# check the updated data
sdg_national_df_copy.head()

(299039, 8)


,indicator_id,country_id,year,value,indicator,country,indicator_category,data_source
0,ADMI.ENDOFLOWERSEC.MAT,AGO,2014,0.0,Administration of a nationally-representative...,Angola,Education,SDG_National
1,ADMI.ENDOFLOWERSEC.MAT,AGO,2015,0.0,Administration of a nationally-representative...,Angola,Education,SDG_National
2,ADMI.ENDOFLOWERSEC.MAT,AGO,2016,0.0,Administration of a nationally-representative...,Angola,Education,SDG_National
3,ADMI.ENDOFLOWERSEC.MAT,AGO,2017,0.0,Administration of a nationally-representative...,Angola,Education,SDG_National
4,ADMI.ENDOFLOWERSEC.MAT,AGO,2018,0.0,Administration of a nationally-representative...,Angola,Education,SDG_National


### Cleaning up SDG Data - Regional

In [16]:
# quick check up and summary of the ilostat data
sdg_regional_df_copy = sdg_regional_df.copy()
sdg_rows = len(sdg_regional_df_copy)
print(sdg_rows)
sdg_regional_df_copy.head()

6731938


,INDICATOR_ID,REGION_ID,YEAR,VALUE,MAGNITUDE,QUALIFIER,INDICATOR_LABEL_EN,COUNTRY_ID,COUNTRY_NAME_EN
0,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,NaN,UIS_EST,Gross intake ratio to the last grade of primar...,COM,Comoros
1,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,NaN,UIS_EST,Gross intake ratio to the last grade of primar...,DJI,Djibouti
2,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,NaN,UIS_EST,Gross intake ratio to the last grade of primar...,SOM,Somalia
3,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,NaN,UIS_EST,Gross intake ratio to the last grade of primar...,SDN,Sudan
4,AIR.1.GLAST,ALECSO: Least developed countries,1999,53.392540,NaN,UIS_EST,Gross intake ratio to the last grade of primar...,COM,Comoros


In [17]:
# standardizing the column names
sdg_regional_df_copy = sdg_regional_df_copy.rename(columns={
    'COUNTRY_NAME_EN': 'country',
    'YEAR': 'year',
    'VALUE': 'value',
    'INDICATOR_LABEL_EN': 'indicator',
    'INDICATOR_ID': 'indicator_id',
    'COUNTRY_ID': 'country_id',
    'MAGNITUDE': 'magnitude',
    'QUALIFIER': 'qualifier',
    'REGION_ID': 'region_id'
})

In [18]:
# converting columns to their appropriate data types
sdg_regional_df_copy['year'] = pd.to_numeric(sdg_regional_df_copy['year'], errors='coerce')
sdg_regional_df_copy['value'] = pd.to_numeric(sdg_regional_df_copy['value'], errors='coerce')
sdg_regional_df_copy['year'] = sdg_regional_df_copy['year'].astype(int)
sdg_regional_df_copy['country'] = sdg_regional_df_copy['country'].str.strip()
sdg_regional_df_copy['indicator_category'] = 'Education'
sdg_regional_df_copy['data_source'] = 'SDG_Regional'

In [19]:
# checking out the shape and missing values of the data
print(sdg_regional_df_copy.shape)
print(sdg_regional_df_copy.isnull().sum())
# checking the percentage of missing values
print(sdg_regional_df_copy.isnull().sum() / len(sdg_regional_df_copy) * 100)

(6731938, 11)
indicator_id                0
region_id                   0
year                        0
value                     352
magnitude             6730882
qualifier             2167701
indicator                   0
country_id                  0
country                     0
indicator_category          0
data_source                 0
dtype: int64
indicator_id           0.000000
region_id              0.000000
year                   0.000000
value                  0.005229
magnitude             99.984314
qualifier             32.200252
indicator              0.000000
country_id             0.000000
country                0.000000
indicator_category     0.000000
data_source            0.000000
dtype: float64


In [20]:
# drop columns that have more than 50% missing values
sdg_regional_df_copy = sdg_regional_df_copy.dropna(thresh=len(sdg_regional_df_copy) * 0.5, axis=1)
# drop subset of missing rows based on the "value" column
sdg_regional_df_copy = sdg_regional_df_copy.dropna(subset=['value'])
# checking the shape of the data
print(sdg_regional_df_copy.shape)
# check the updated data
sdg_regional_df_copy.head()

(6731586, 10)


,indicator_id,region_id,year,value,qualifier,indicator,country_id,country,indicator_category,data_source
0,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,UIS_EST,Gross intake ratio to the last grade of primar...,COM,Comoros,Education,SDG_Regional
1,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,UIS_EST,Gross intake ratio to the last grade of primar...,DJI,Djibouti,Education,SDG_Regional
2,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,UIS_EST,Gross intake ratio to the last grade of primar...,SOM,Somalia,Education,SDG_Regional
3,AIR.1.GLAST,ALECSO: Least developed countries,1998,53.372169,UIS_EST,Gross intake ratio to the last grade of primar...,SDN,Sudan,Education,SDG_Regional
4,AIR.1.GLAST,ALECSO: Least developed countries,1999,53.392540,UIS_EST,Gross intake ratio to the last grade of primar...,COM,Comoros,Education,SDG_Regional


### Cleaning the World Bank Data

In [21]:
# quick check up and summary of the ilostat data
worldbank_df
worldbank_df_copy = worldbank_df.copy()
worldbank_rows = len(worldbank_df_copy)
print(worldbank_rows)
worldbank_df_copy.head()

1350


,CountryName,CountryCode,Year,"Foreign direct investment, net inflows (% of GDP)","Agriculture, forestry, and fishing, value added (% of GDP)","Industry (including construction), value added (% of GDP)","Services, value added (% of GDP)",GDP growth (annual %),"Literacy rate, youth total (% of people ages 15-24)","Literacy rate, adult total (% of people ages 15 and above)",...,"School enrollment, primary (% gross)","School enrollment, secondary (% gross)","School enrollment, tertiary (% gross)","Government expenditure on education, total (% of government expenditure)","Government expenditure on education, total (% of GDP)","Unemployment, youth total (% of total labor force ages 15-24)",Unemployment with advanced education (% of total labor force with advanced education),"Unemployment, total (% of total labor force)",Population ages 15-64 (% of total population),Population growth (annual %)
0,Algeria,DZA,2000,0.511221,8.395048,53.330970,33.110546,3.8,NaN,NaN,...,109.734398,64.595421,NaN,NaN,NaN,50.641,NaN,29.77,61.911421,1.399669
1,Algeria,DZA,2001,1.873492,8.886180,40.814590,43.993001,3.0,NaN,NaN,...,109.017838,68.763298,15.294780,NaN,NaN,46.921,NaN,27.30,62.939440,1.373291
2,Algeria,DZA,2002,1.731189,8.413026,39.766715,44.684560,5.4,90.139999,69.870003,...,110.677277,72.583069,16.975420,NaN,NaN,46.026,NaN,25.90,63.916002,1.330395
3,Algeria,DZA,2003,0.868037,8.456720,42.040102,43.000036,6.5,NaN,NaN,...,111.387787,75.435867,18.211821,NaN,NaN,43.872,NaN,23.72,64.813548,1.329615
4,Algeria,DZA,2004,0.962587,7.722174,44.981463,41.318486,4.5,NaN,NaN,...,111.655128,79.629860,18.870930,NaN,NaN,34.204,22.83,17.65,65.597250,1.396440


In [22]:
# Standardize column names
worldbank_df_copy = worldbank_df_copy.rename(columns={
    'CountryName': 'country',
    'CountryCode': 'country_id',
    'Year': 'year'
})

In [23]:
# checking out the shape and missing values of the data
print(worldbank_df_copy.shape)
print(worldbank_df_copy.isnull().sum())
# checking the percentage of missing values
print(worldbank_df_copy.isnull().sum() / len(worldbank_df_copy) * 100)

(1350, 22)
country                                                                                     0
country_id                                                                                  0
year                                                                                        0
Foreign direct investment, net inflows (% of GDP)                                          49
Agriculture, forestry, and fishing, value added (% of GDP)                                 99
Industry (including construction), value added (% of GDP)                                 110
Services, value added (% of GDP)                                                          119
GDP growth (annual %)                                                                      45
Literacy rate, youth total (% of people ages 15-24)                                      1075
Literacy rate, adult total (% of people ages 15 and above)                               1108
Primary completion rate, total (% of relevant age

it'd be totally normal to drop columns with more than 50% missing values but majority of those columns are relevant to our analysis so we'll keep them and fill them up by using linear interporlation grouped by country and ordered by year. This method will fill in missing values based on the linear trend between the available data points for that specific country.

In [24]:
# Convert year to integer
worldbank_df_copy['year'] = pd.to_numeric(worldbank_df_copy['year'], errors='coerce')
worldbank_df_copy = worldbank_df_copy.dropna(subset=['year'])
worldbank_df_copy['year'] = worldbank_df_copy['year'].astype(int)

In [25]:
# Identify numerical columns for imputation (excluding 'Year')
wrlbk_num_cols = worldbank_df_copy.select_dtypes(include=np.number).columns.tolist()
if 'year' in worldbank_df_copy:
    wrlbk_num_cols.remove('year')

print(f"Numerical columns to be imputed: {len(worldbank_df_copy)}")

# --- Imputation Strategy: Linear Interpolation per Country ---
# Using transform is safer than apply for returning a result with the same index.

def impute_by_country_transform(df, cols_to_impute):
    """
    Applies linear interpolation to specified columns, grouped by CountryCode,
    using the transform method for correct index alignment.
    """
    print("Starting imputation (Linear Interpolation) using transform...")
    
    # Create a copy to work on
    df_cleaned = df.copy()
    
    # Sort by CountryCode and Year to ensure correct time-series order
    df_cleaned = df_cleaned.sort_values(by=['country_id', 'year']).reset_index(drop=True)
    
    # Apply linear interpolation to each column individually using transform
    for col in cols_to_impute:
        # Interpolate within each country group. 'limit_direction="both"' ensures
        # forward-fill and backward-fill are applied to fill gaps at the start/end
        # of the time series where possible.
        df_cleaned[col] = df_cleaned.groupby('country_id')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )
        
    return df_cleaned

# Perform the imputation
worldbank_df_copy = impute_by_country_transform(worldbank_df_copy, wrlbk_num_cols)

Numerical columns to be imputed: 1350
Starting imputation (Linear Interpolation) using transform...


In [26]:
# --- Final Check and Save ---

# Calculate missing value percentages after cleaning
missing_percent_after = worldbank_df_copy.isnull().sum() * 100 / len(worldbank_df_copy)
missing_df_after = pd.DataFrame({
    'Column Name': missing_percent_after.index, 
    'Missing Percentage After Imputation': missing_percent_after.values
})
missing_df_after = missing_df_after.sort_values(by='Missing Percentage After Imputation', ascending=False)

print("\n--- Missing Value Percentages After Imputation (Columns with >0% missing) ---")
print(missing_df_after[missing_df_after['Missing Percentage After Imputation'] > 0].to_markdown(index=False))

# # Save the cleaned data
# df_cleaned.to_csv(output_file_path, index=False)
# print(f"\nCleaned dataset saved to: {output_file_path}")


--- Missing Value Percentages After Imputation (Columns with >0% missing) ---
| Column Name                                                                           |   Missing Percentage After Imputation |
|:--------------------------------------------------------------------------------------|--------------------------------------:|
| Unemployment with advanced education (% of total labor force with advanced education) |                              11.1111  |
| Primary completion rate, total (% of relevant age group)                              |                               5.55556 |
| Pupil-teacher ratio in primary education (headcount basis)                            |                               3.7037  |
| Government expenditure on education, total (% of GDP)                                 |                               3.7037  |
| Services, value added (% of GDP)                                                      |                               3.7037  |
| Literacy 

We still have little missing values left after our major cleaning, we'll use a minor median imputation to implement this clean up

In [27]:
def final_median_imputation(df, cols_to_impute):
    """
    Applies global median imputation to all remaining missing values in the specified columns.
    """
    print("Starting final imputation (Global Median)...")
    df_final = df.copy()
    
    for col in cols_to_impute:
        # Calculate the median of the non-missing values in the entire column
        median_value = df_final[col].median()
        
        # Fill remaining NaNs with the calculated median
        df_final[col] = df_final[col].fillna(median_value)
        
    return df_final

# Perform the final imputation
worldbank_df_copy = final_median_imputation(worldbank_df_copy, wrlbk_num_cols )

Starting final imputation (Global Median)...


In [28]:
# --- Final Check ---

# Calculate missing value percentages after final cleaning
missing_percent_after = worldbank_df_copy.isnull().sum() * 100 / len(worldbank_df_copy)
missing_df_after = pd.DataFrame({
    'Column Name': missing_percent_after.index, 
    'Missing Percentage After Final Imputation': missing_percent_after.values
})
missing_df_after = missing_df_after.sort_values(by='Missing Percentage After Final Imputation', ascending=False)

print("\n--- Missing Value Percentages After Final Imputation ---")
# Should show 0% for all columns
print(missing_df_after[missing_df_after['Missing Percentage After Final Imputation'] > 0].to_markdown(index=False))
if missing_df_after['Missing Percentage After Final Imputation'].max() == 0:
    print("All missing values have been successfully imputed.")


--- Missing Value Percentages After Final Imputation ---
| Column Name   | Missing Percentage After Final Imputation   |
|---------------|---------------------------------------------|
All missing values have been successfully imputed.


In [29]:
# Standardize country names
worldbank_df_copy['country'] = worldbank_df_copy['country'].str.strip()

# Add data source
worldbank_df_copy['data_source'] = 'WorldBank'

In [30]:
# summary report of the cleaning
print(f"\n✓ Cleaned World Bank data: {worldbank_rows} → {len(worldbank_df_copy)} rows")
print(f"   • Countries: {worldbank_df_copy['country'].nunique()}")
print(f"   • Year range: {worldbank_df_copy['year'].min()} - {worldbank_df_copy['year'].max()}")
print(f"   • Indicators: {len([col for col in worldbank_df_copy.columns if '%' in col or 'rate' in col.lower()])}")


✓ Cleaned World Bank data: 1350 → 1350 rows
   • Countries: 54
   • Year range: 2000 - 2024
   • Indicators: 18


The ILOStat dataset and the World Bank datasets are similar in that they both contain information about labor market indicators, such as employment rates, unemployment rates, and labor force participation rates. The SDG datasets, on the other hand, contain information about education indicators, such as enrollment rates, literacy rates, and school completion rates.

So for this reason, the ILOStat data and the World bank data will be combined for better analysis.

In [31]:
worldbank_df_cleaned = worldbank_df_copy
sdg_national_df_cleaned = sdg_national_df_copy
sdg_regional_df_cleaned = sdg_regional_df_copy
ilostat_df_cleaned = ilostat_df_copy

In [32]:
def harmonize_country_names(ilostat_df, sdg_national_df, sdg_regional_df, worldbank_df):

    # Country mappings
    country_mappings = {
        "Côte d'Ivoire": "Cote d'Ivoire",
        "Democratic Republic of the Congo": "Congo, Dem. Rep.",
        "Congo": "Congo, Rep.",
        "Egypt": "Egypt, Arab Rep.",
        "Gambia": "Gambia, The",
        "Cabo Verde": "Cape Verde"
    }

    # Apply mappings
    datasets = [ilostat_df, sdg_national_df, sdg_regional_df, worldbank_df]

    updated_datasets = []
    for df in datasets:
        if df is not None and 'country' in df.columns:
            df = df.copy()
            df['country'] = df['country'].replace(country_mappings)
        updated_datasets.append(df)

    # Extract updated versions
    ilo_df, nat_df, reg_df, wb_df = updated_datasets

    # Collect unique countries
    countries_ilostat = set(ilo_df['country'].unique()) if ilo_df is not None else set()
    countries_sdg_nat = set(nat_df['country'].unique()) if nat_df is not None else set()
    countries_sdg_reg = set(reg_df['country'].unique()) if reg_df is not None else set()
    countries_wb = set(wb_df['country'].unique()) if wb_df is not None else set()

    all_countries = countries_ilostat | countries_sdg_nat | countries_sdg_reg | countries_wb

    # Print summary
    print(f"\n✓ Country harmonization complete")
    print(f"   • Total unique countries: {len(all_countries)}")
    print(f"   • ILOSTAT: {len(countries_ilostat)}")
    print(f"   • SDG National: {len(countries_sdg_nat)}")
    print(f"   • SDG Regional: {len(countries_sdg_reg)}")
    print(f"   • World Bank: {len(countries_wb)}")

    metadata = {
        'countries': sorted(list(all_countries))
    }

    return ilo_df, nat_df, reg_df, wb_df, metadata

In [33]:
# applying the harmonization function
ilostat_df_cleaned, sdg_nat_df_cleaned, sdg_reg_df_cleaned, worldbank_df_cleaned, metadata = harmonize_country_names(
    ilostat_df_cleaned,
    sdg_national_df_cleaned,
    sdg_regional_df_cleaned,
    worldbank_df_cleaned
)


✓ Country harmonization complete
   • Total unique countries: 56
   • ILOSTAT: 51
   • SDG National: 54
   • SDG Regional: 54
   • World Bank: 54


### Combining the Datasets (ILOStat and World Bank and SDG) Together

In [34]:
print("CREATING INTEGRATED DATASET")
# Start with World Bank data as base (most comprehensive)
base_df = worldbank_df_cleaned[['country', 'year', 'country_id']].drop_duplicates()

print(f"\n[1/5] Base structure: {len(base_df)} country-year combinations")

# Aggregate ILOSTAT data by country and year
if ilostat_df_cleaned is not None:
    ilo_pivot = ilostat_df_cleaned.pivot_table(
        index=['country', 'year'],
        columns='indicator',
        values='value',
        aggfunc='mean'
    ).reset_index()
    
    # Rename columns to be more descriptive
    ilo_pivot.columns = ['country', 'year'] + [f'ILO_{col}' for col in ilo_pivot.columns[2:]]
    
    base_df = base_df.merge(ilo_pivot, on=['country', 'year'], how='left')
    print(f"[2/5] Merged ILOSTAT data: {len(base_df)} rows, {len(ilo_pivot.columns)-2} labour indicators")

# Merge SDG National data
if sdg_nat_df_cleaned is not None and len(sdg_nat_df_cleaned) > 0:
    sdg_nat_pivot = sdg_nat_df_cleaned.pivot_table(
        index=['country', 'year'],
        columns='indicator_id',
        values='value',
        aggfunc='mean'
    ).reset_index()
    
    # Rename columns with SDG_NAT prefix
    sdg_nat_pivot.columns = ['country', 'year'] + [f'SDG_NAT_{col}' for col in sdg_nat_pivot.columns[2:]]
    
    base_df = base_df.merge(sdg_nat_pivot, on=['country', 'year'], how='left')
    print(f"[3/5] Merged SDG National data: {len(base_df)} rows, {len(sdg_nat_pivot.columns)-2} education indicators")

# Merge World Bank indicators
wb_indicators = [col for col in worldbank_df_cleaned.columns 
                if col not in ['country', 'year', 'country_id', 'data_source']]

wb_subset = worldbank_df_cleaned[['country', 'year'] + wb_indicators]
base_df = base_df.merge(wb_subset, on=['country', 'year'], how='left')

print(f"[5/5] Merged World Bank data: {len(base_df)} rows, {len(wb_indicators)} indicators")

integrated_df = base_df

CREATING INTEGRATED DATASET

[1/5] Base structure: 1350 country-year combinations
[2/5] Merged ILOSTAT data: 1350 rows, 5 labour indicators
[3/5] Merged SDG National data: 1350 rows, 2360 education indicators
[5/5] Merged World Bank data: 1350 rows, 19 indicators


In [35]:
# report for merged dataset
print(f"\n✓ Integrated dataset created:")
print(f"   • Total rows: {len(base_df)}")
print(f"   • Total columns: {len(base_df.columns)}")
print(f"   • Year range: {base_df['year'].min()} - {base_df['year'].max()}")
print(f"   • Countries: {base_df['country'].nunique()}")


✓ Integrated dataset created:
   • Total rows: 1350
   • Total columns: 2387
   • Year range: 2000 - 2024
   • Countries: 54


### cleaning the merged data

In [36]:
# create a copy of the data
base_df_copy = base_df.copy()
# Calculate missing percentage for each column
missing_pct = (base_df_copy.isnull().sum() / len(base_df_copy) * 100).sort_values(ascending=False)

print(f"\nColumns with >50% missing values:")
high_missing = missing_pct[missing_pct > 50]
for col, pct in high_missing.head(10).items():
    print(f"   • {col}: {pct:.1f}%")


Columns with >50% missing values:
   • SDG_NAT_ICTSKILLSCRTY.AG75OROVER.F: 99.9%
   • SDG_NAT_ICTSKILLONLCNS.AG75OROVER.GPIA: 99.9%
   • SDG_NAT_GAR.5T8.URB.Q3.GPIA: 99.9%
   • SDG_NAT_GAR.5T8.URB.Q4.GPIA: 99.9%
   • SDG_NAT_GAR.5T8.URB.Q5.GPIA: 99.9%
   • SDG_NAT_ICTSKILLFONLCRS.AG75OROVER.F: 99.9%
   • SDG_NAT_ICTSKILLINTBNK.AGUNDER15Y.GPIA: 99.9%
   • SDG_NAT_ICTSKILLONLCNS.AG75OROVER.F: 99.9%
   • SDG_NAT_ICTSKILLONLCNS.AGUNDER15Y.GPIA: 99.9%
   • SDG_NAT_GAR.5T8.RUR.Q4.GPIA: 99.9%


In [37]:
# Drop all columns that have more than 50% missing values
base_df_copy = base_df_copy.dropna(thresh=len(base_df_copy)*0.5, axis=1)

In [38]:
base_df_copy.columns

Index(['country', 'year', 'country_id', 'SDG_NAT_AIR.1.GLAST',
       'SDG_NAT_AIR.1.GLAST.F', 'SDG_NAT_AIR.1.GLAST.GPIA',
       'SDG_NAT_AIR.1.GLAST.M', 'SDG_NAT_AIR.2.GPV.GLAST',
       'SDG_NAT_AIR.2.GPV.GLAST.F', 'SDG_NAT_AIR.2.GPV.GLAST.GPIA',
       'SDG_NAT_AIR.2.GPV.GLAST.M', 'SDG_NAT_CR.MOD.1', 'SDG_NAT_CR.MOD.1.F',
       'SDG_NAT_CR.MOD.1.GPIA', 'SDG_NAT_CR.MOD.1.M', 'SDG_NAT_CR.MOD.2',
       'SDG_NAT_CR.MOD.2.F', 'SDG_NAT_CR.MOD.2.GPIA', 'SDG_NAT_CR.MOD.2.M',
       'SDG_NAT_CR.MOD.3', 'SDG_NAT_CR.MOD.3.F', 'SDG_NAT_CR.MOD.3.GPIA',
       'SDG_NAT_CR.MOD.3.M', 'SDG_NAT_GER.5T8', 'SDG_NAT_OAEPG.1',
       'SDG_NAT_OAEPG.1.F', 'SDG_NAT_OAEPG.1.GPIA', 'SDG_NAT_OAEPG.1.M',
       'SDG_NAT_ODAFLOW.VOLUMESCHOLARSHIP', 'SDG_NAT_ROFST.1.CP',
       'SDG_NAT_ROFST.MOD.1', 'SDG_NAT_ROFST.MOD.1.F',
       'SDG_NAT_ROFST.MOD.1.GPIA', 'SDG_NAT_ROFST.MOD.1.M',
       'SDG_NAT_ROFST.MOD.2', 'SDG_NAT_ROFST.MOD.2.F',
       'SDG_NAT_ROFST.MOD.2.GPIA', 'SDG_NAT_ROFST.MOD.2.M',
       'SDG_

The columns with significantly high missing values are quite significant in the base_df and for the analysis, we'll hold off on deleting them at this point. Although if they are numeric and the missing values is less than 30%, we'll be filling them with interpolated values.

For context, Interporlation is the filling of missing numeric values by estimating them based on nearby known values, instead of leaving them empty or replacing them with a constant.

In our case, we'll be interpolating by estimating missing numeric values using trends from nearby years, separately for each country.

In [39]:
# 1. Identify numeric columns to clean (skip keys)
base_numeric_cols = [
    col for col in base_df_copy.select_dtypes(include=[np.number]).columns
    if col not in ('year')
]

print(f"Numeric indicators to impute: {len(base_numeric_cols)}")


# 2. Reuse the interpolation helper (change group key to Country)
def impute_by_country_transform(df, cols_to_impute, country_col='country'):
    print("Starting imputation (linear interpolation by country)...")
    df_cleaned = df.sort_values([country_col, 'year']).reset_index(drop=True)

    for col in cols_to_impute:
        df_cleaned[col] = (
            df_cleaned.groupby(country_col)[col]
            .transform(lambda x: x.interpolate(method='linear', limit_direction='both'))
        )
    return df_cleaned

base_df_copy = impute_by_country_transform(base_df_copy, base_numeric_cols, country_col='country')

# 3. Apply the global median pass to mop up columns with entire gaps
def final_median_imputation(df, cols_to_impute):
    print("Starting final imputation (Global Median)...")
    df_final = df.copy()
    for col in cols_to_impute:
        median_value = df_final[col].median()
        df_final[col] = df_final[col].fillna(median_value)
    return df_final

base_df_copy = final_median_imputation(base_df_copy, base_numeric_cols)

# 4. Optional diagnostics
missing_pct = base_df_copy[base_numeric_cols].isnull().mean() * 100
still_missing = missing_pct[missing_pct > 0]
if still_missing.empty:
    print("✓ Integrated dataset numeric columns fully imputed")
else:
    print("⚠ Columns still missing after cleanup:")
    print(still_missing)

Numeric indicators to impute: 64
Starting imputation (linear interpolation by country)...
Starting final imputation (Global Median)...
✓ Integrated dataset numeric columns fully imputed


### Dealing with Outliers

In [40]:
# selecting the numeric columns
numeric_cols = [col for col in base_numeric_cols if col not in ['year', 'country_id']]

outlier_summary = {}

for col in base_numeric_cols[:10]:  # Check first 10 numeric columns
    if base_df_copy[col].notna().sum() > 0:
        Q1 = base_df_copy[col].quantile(0.25)
        Q3 = base_df_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        outliers = ((base_df_copy[col] < lower_bound) | (base_df_copy[col] > upper_bound)) & base_df_copy[col].notna()
        outlier_count = outliers.sum()
        
        if outlier_count > 0:
            outlier_summary[col] = outlier_count

# report for outlier detection
if outlier_summary:
    print(f"\nOutliers detected (using 3*IQR method):")
    for col, count in sorted(outlier_summary.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"   • {col}: {count} outliers")
else:
    print("\n✓ No significant outliers detected")

metadata['outliers'] = outlier_summary


Outliers detected (using 3*IQR method):
   • SDG_NAT_AIR.2.GPV.GLAST.M: 1 outliers


In [41]:
from sklearn.preprocessing import RobustScaler

def treat_outliers(df, columns, method="winsorize", iqr_factor=3, cap_quantiles=(0.01, 0.99)):
    df_out = df.copy()
    for col in columns:
        if df_out[col].notna().sum() == 0:
            continue

        q1 = df_out[col].quantile(0.25)
        q3 = df_out[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - iqr_factor * iqr
        upper = q3 + iqr_factor * iqr

        if method == "winsorize":
            df_out[col] = df_out[col].clip(lower=lower, upper=upper)
        elif method == "capping":
            low_cap = df_out[col].quantile(cap_quantiles[0])
            high_cap = df_out[col].quantile(cap_quantiles[1])
            df_out[col] = df_out[col].clip(lower=low_cap, upper=high_cap)
        elif method == "flag":
            df_out[f"{col}_is_outlier"] = ((df_out[col] < lower) | (df_out[col] > upper)).astype(int)
    return df_out

final_base_df = treat_outliers(base_df_copy, outlier_summary.keys(), method="winsorize")

For Easier Analysis, we'll be creating feature (column) categories from the base_df so that we cana ealiy identify indicators that the features are related to

In [42]:
# def create_feature_categories(integrated_df, metadata=None, verbose=True):
#     """Categorize columns into thematic indicator groups."""
#     if integrated_df is None or integrated_df.empty:
#         if verbose:
#             print("⚠ No integrated dataset to process")
#         return {}

#     if metadata is None:
#         metadata = {}

#     cols = integrated_df.columns.tolist()

#     def pick_by_keywords(keywords):
#         return [
#             col for col in cols
#             if any(keyword in col.lower() for keyword in keywords)
#         ]

#     feature_categories = {
#         "education": pick_by_keywords(
#             ['education', 'school', 'literacy', 'enrollment',
#              'pupil', 'teacher', 'primary', 'secondary',
#              'tertiary', 'completion']
#         ),
#         "labour": pick_by_keywords(
#             ['employment', 'unemployment', 'labour', 'labor', 'ilo_', 'workforce']
#         ),
#         "economic": pick_by_keywords(
#             ['gdp', 'fdi', 'investment', 'agriculture', 'industry',
#              'services', 'growth', 'value added']
#         ),
#         "demographic": pick_by_keywords(['population', 'ages'])
#     }

#     metadata['feature_categories'] = feature_categories

#     if verbose:
#         print("\n✓ Feature categorization complete:")
#         for key, vals in feature_categories.items():
#             print(f"   • {key.capitalize()} indicators: {len(vals)}")

#     return feature_categories

In [43]:
def create_feature_categories(integrated_df, metadata=None, verbose=True):
    """Categorize columns into thematic indicator groups."""
    if integrated_df is None or integrated_df.empty:
        if verbose:
            print("⚠ No integrated dataset to process")
        return {}

    cols = integrated_df.columns.tolist()

    education_keywords = ['education','school','literacy','enrollment','pupil','teacher',
                          'primary','secondary','tertiary','completion','sdg_nat','sdg_reg']
    labour_keywords = ['employment','unemployment','labour','labor','ilo_','workforce']
    economic_keywords = ['gdp','fdi','investment','agriculture','industry','services','growth','value added']
    demographic_keywords = ['population','ages']

    def bucket(keywords):
        return [col for col in cols if any(k in col.lower() for k in keywords)]

    feature_categories = {
        'education': bucket(education_keywords),
        'labour': bucket(labour_keywords),
        'economic': bucket(economic_keywords),
        'demographic': bucket(demographic_keywords)
    }

    if metadata is not None:
        metadata['feature_categories'] = feature_categories

    if verbose:
        print("\n✓ Feature categorization complete:")
        for key, values in feature_categories.items():
            print(f"   • {key.capitalize()} indicators: {len(values)}")

    return feature_categories

In [44]:
feature_categories = create_feature_categories(final_base_df, metadata)


✓ Feature categorization complete:
   • Education indicators: 55
   • Labour indicators: 3
   • Economic indicators: 8
   • Demographic indicators: 5


### Saving the Final Data

In [45]:
def save_processed_data(
    ilostat_df=None,
    sdg_national_df=None,
    sdg_regional_df=None,
    worldbank_df=None,
    integrated_df=None,
    metadata=None,
    data_dir=".",
    output_subdir="processed",
):
    """
    Save all processed datasets and metadata to disk.
    """
    print("\n" + "=" * 80)
    print("SAVING PROCESSED DATA")
    print("=" * 80)

    output_dir = Path(data_dir) / output_subdir
    output_dir.mkdir(parents=True, exist_ok=True)

    def _save_csv(df, name):
        df.to_csv(output_dir / f"{name}.csv", index=False)
        print(f"✓ Saved: {name}.csv ({len(df)} rows)")

    if ilostat_df is not None:
        _save_csv(ilostat_df, "ilostat_cleaned")

    if sdg_national_df is not None:
        _save_csv(sdg_national_df, "sdg_national_cleaned")

    if sdg_regional_df is not None:
        _save_csv(sdg_regional_df, "sdg_regional_cleaned")

    if worldbank_df is not None:
        _save_csv(worldbank_df, "worldbank_cleaned")

    if integrated_df is not None:
        _save_csv(integrated_df, "integrated_data")

    def convert_to_serializable(obj):
        if isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [convert_to_serializable(v) for v in obj]
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        if isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        if isinstance(obj, (str, int, float, bool, type(None))):
            return obj
        return str(obj)

    if metadata is None:
        metadata = {}

    metadata_clean = convert_to_serializable(metadata)
    with open(output_dir / "metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata_clean, f, indent=2)
    print("✓ Saved: metadata.json")

    print(f"\n✓ All processed data saved to: {output_dir}")

    return output_dir

In [46]:
save_processed_data(
    ilostat_df=ilostat_df_cleaned,
    sdg_national_df=sdg_nat_df_cleaned,
    sdg_regional_df=sdg_reg_df_cleaned,
    worldbank_df=worldbank_df_cleaned,
    integrated_df=final_base_df,
    metadata=metadata,
    data_dir=".",
)


SAVING PROCESSED DATA
✓ Saved: ilostat_cleaned.csv (674384 rows)
✓ Saved: sdg_national_cleaned.csv (299039 rows)
✓ Saved: sdg_regional_cleaned.csv (6731586 rows)
✓ Saved: worldbank_cleaned.csv (1350 rows)
✓ Saved: integrated_data.csv (1350 rows)
✓ Saved: metadata.json

✓ All processed data saved to: processed


WindowsPath('processed')

## Analysis

In [47]:
# Load integrated dataset and metadata
df = pd.read_csv(data_dir / 'integrated_data.csv')

with open(data_dir / 'metadata.json', 'r') as f:
    metadata = json.load(f)

# Identify SDG columns
sdg_columns = [col for col in df.columns if col.startswith('SDG_')]

# Initialize results dictionary
results = {}

print(f"Loaded integrated dataset: {df.shape}")
print(f"Identified {len(sdg_columns)} SDG indicators")

Loaded integrated dataset: (1350, 67)
Identified 45 SDG indicators


### SDG Indicator Analysis

In [48]:
print("SDG INDICATOR ANALYSIS (NEW SECTION)")
print(f"\n[1/4] SDG DATA OVERVIEW")
print(f"Total SDG indicators: {len(sdg_columns)}")

# Data availability
sdg_availability = {}
for col in sdg_columns:
    non_null_count = df[col].notna().sum()
    non_null_pct = (non_null_count / len(df)) * 100
    sdg_availability[col] = {
        'count': non_null_count,
        'percentage': non_null_pct
    }

# Select SDG indicators with sufficient data
sdg_with_data = {k: v for k, v in sdg_availability.items() if v['percentage'] > 10}
print(f"SDG indicators with >10% data: {len(sdg_with_data)}")

SDG INDICATOR ANALYSIS (NEW SECTION)

[1/4] SDG DATA OVERVIEW
Total SDG indicators: 45
SDG indicators with >10% data: 45


In [49]:
# Show top indicators
print("\nTop SDG indicators by data availability:")
sorted_sdg = sorted(sdg_availability.items(), key=lambda x: x[1]['percentage'], reverse=True)
for col, info in sorted_sdg[:10]:
    print(f"   {col[:55]:<55} : {info['percentage']:>5.1f}%")

print(f"\n[2/4] SDG INDICATOR STATISTICS")

# Key SDG indicators - select those with most data
key_sdg = [col for col in sdg_columns if df[col].notna().sum() > len(df) * 0.1][:10]

sdg_stats = {}
for col in key_sdg:
    if df[col].notna().sum() > 0:
        sdg_stats[col] = {
            'mean': float(df[col].mean()),
            'median': float(df[col].median()),
            'std': float(df[col].std()),
            'min': float(df[col].min()),
            'max': float(df[col].max()),
            'records': int(df[col].notna().sum())
        }

print(f"Statistics for {len(sdg_stats)} key SDG indicators:")
for col, stats_info in list(sdg_stats.items())[:5]:
    print(f"\n   {col[:50]}:")
    print(f"      Mean: {stats_info['mean']:.2f}, Median: {stats_info['median']:.2f}")
    print(f"      Range: [{stats_info['min']:.2f}, {stats_info['max']:.2f}]")
    print(f"      Records: {stats_info['records']}")


Top SDG indicators by data availability:
   SDG_NAT_AIR.1.GLAST                                     : 100.0%
   SDG_NAT_AIR.1.GLAST.F                                   : 100.0%
   SDG_NAT_AIR.1.GLAST.GPIA                                : 100.0%
   SDG_NAT_AIR.1.GLAST.M                                   : 100.0%
   SDG_NAT_AIR.2.GPV.GLAST                                 : 100.0%
   SDG_NAT_AIR.2.GPV.GLAST.F                               : 100.0%
   SDG_NAT_AIR.2.GPV.GLAST.GPIA                            : 100.0%
   SDG_NAT_AIR.2.GPV.GLAST.M                               : 100.0%
   SDG_NAT_CR.MOD.1                                        : 100.0%
   SDG_NAT_CR.MOD.1.F                                      : 100.0%

[2/4] SDG INDICATOR STATISTICS
Statistics for 10 key SDG indicators:

   SDG_NAT_AIR.1.GLAST:
      Mean: 67.73, Median: 67.04
      Range: [15.69, 156.17]
      Records: 1350

   SDG_NAT_AIR.1.GLAST.F:
      Mean: 65.91, Median: 65.52
      Range: [12.19, 167.04]
      Record

In [50]:
print(f"\n[3/4] SDG-LABOUR CORRELATION ANALYSIS")
# Get labour indicators
labour_cols = [col for col in df.columns if 'unemployment' in col.lower() 
                and df[col].notna().sum() > 50][:3]

if not labour_cols:
    print("   WARNING: No labour indicators found")
    sdg_labour_corr = []
else:
    sdg_labour_corr = []
    
    # Correlate SDG with labour
    for sdg_col in key_sdg[:10]:  # Top 10 SDG indicators
        for labour_col in labour_cols:
            valid_data = df[[sdg_col, labour_col]].dropna()
            
            if len(valid_data) >= 30:
                try:
                    corr, p_value = pearsonr(valid_data[sdg_col], valid_data[labour_col])
                    
                    sdg_labour_corr.append({
                        'sdg_indicator': sdg_col,
                        'labour_indicator': labour_col,
                        'correlation': corr,
                        'p_value': p_value,
                        'sample_size': len(valid_data),
                        'significance': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
                    })
                except Exception as e:
                    print(f"   WARNING: Correlation failed for {sdg_col[:30]} vs {labour_col[:30]}: {str(e)}")

    if sdg_labour_corr:
        # Sort by absolute correlation
        sdg_labour_corr.sort(key=lambda x: abs(x['correlation']), reverse=True)
        
        print(f"Computed {len(sdg_labour_corr)} SDG-Labour correlation pairs")
        print("\nTop 5 SDG-Labour correlations:")
        for i, result in enumerate(sdg_labour_corr[:5], 1):
            print(f"\n   [{i}] {result['sdg_indicator'][:40]}")
            print(f"       -> {result['labour_indicator'][:40]}")
            print(f"       r = {result['correlation']:.3f} (p={result['p_value']:.4f}) {result['significance']}")
            print(f"       n = {result['sample_size']}")


[3/4] SDG-LABOUR CORRELATION ANALYSIS
Computed 30 SDG-Labour correlation pairs

Top 5 SDG-Labour correlations:

   [1] SDG_NAT_CR.MOD.1
       -> Unemployment, youth total (% of total la
       r = 0.483 (p=0.0000) ***
       n = 1350

   [2] SDG_NAT_CR.MOD.1.F
       -> Unemployment, youth total (% of total la
       r = 0.476 (p=0.0000) ***
       n = 1350

   [3] SDG_NAT_CR.MOD.1.F
       -> Unemployment, total (% of total labor fo
       r = 0.437 (p=0.0000) ***
       n = 1350

   [4] SDG_NAT_CR.MOD.1
       -> Unemployment, total (% of total labor fo
       r = 0.428 (p=0.0000) ***
       n = 1350

   [5] SDG_NAT_AIR.2.GPV.GLAST.GPIA
       -> Unemployment, total (% of total labor fo
       r = 0.386 (p=0.0000) ***
       n = 1350


In [51]:
print(f"\n[4/4] SDG COUNTRY-LEVEL ANALYSIS")
print("-" * 40)

# Analyze SDG by country
if 'country' in df.columns or 'Country' in df.columns:
    country_col = 'Country' if 'Country' in df.columns else 'country'
    
    # Select one key SDG indicator
    if key_sdg:
        sample_sdg = key_sdg[0]
        country_sdg = df.groupby(country_col)[sample_sdg].mean().sort_values(ascending=False)
        country_sdg_clean = country_sdg.dropna()
        
        print(f"\nCountry analysis for: {sample_sdg[:50]}")
        print(f"Countries with data: {len(country_sdg_clean)}")
        
        if len(country_sdg_clean) > 0:
            print(f"\nTop 5 countries:")
            for country, value in country_sdg_clean.head(5).items():
                print(f"   {country}: {value:.2f}")

            print(f"\nBottom 5 countries:")
            for country, value in country_sdg_clean.tail(5).items():
                print(f"   {country}: {value:.2f}")

# Save SDG results
results['sdg_analysis'] = {
'availability': sdg_availability,
'statistics': sdg_stats,
'sdg_labour_correlations': sdg_labour_corr if sdg_labour_corr else []
}


[4/4] SDG COUNTRY-LEVEL ANALYSIS
----------------------------------------

Country analysis for: SDG_NAT_AIR.1.GLAST
Countries with data: 54

Top 5 countries:
   Seychelles: 114.16
   Botswana: 99.16
   Algeria: 97.69
   Kenya: 95.42
   Cape Verde: 95.16

Bottom 5 countries:
   Niger: 43.78
   Chad: 34.38
   Central African Republic: 32.67
   Equatorial Guinea: 32.52
   South Sudan: 28.66


### Exploratory Data Analysis (EDA)

In [52]:
# [1/5] Basic statistics
print("\n[1/5] BASIC STATISTICS")
print(f"Total records: {len(df)}")
country_col = 'Country' if 'Country' in df.columns else 'country'
year_col = 'Year' if 'Year' in df.columns else 'year'
print(f"Countries: {df['country'].nunique()}")
print(f"Year range: {df['year'].min()} - {df['year'].max()}")
print(f"Total features: {len(df.columns)}")
print(f"SDG indicators: {len(sdg_columns)}")


[1/5] BASIC STATISTICS
Total records: 1350
Countries: 54
Year range: 2000 - 2024
Total features: 67
SDG indicators: 45


In [53]:
# [2/5] Missing value analysis
print("\n[2/5] MISSING VALUE ANALYSIS")
missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
significant_missing = missing_pct[missing_pct > 0][:10]

if len(significant_missing) > 0:
    print("Top 10 columns with missing values:")
    for col, pct in significant_missing.items():
        print(f"   - {col[:60]}: {pct:.1f}%")
else:
    print("   [OK] No missing values detected")


[2/5] MISSING VALUE ANALYSIS
   [OK] No missing values detected


In [54]:
print("\n[3/5] DISTRIBUTION ANALYSIS")
numeric_cols = df.select_dtypes(include=[np.number]).columns
key_indicators = [
    col for col in numeric_cols
    if any(keyword in col.lower() for keyword in ['unemployment', 'literacy', 'enrollment', 'gdp'])
][:5]

skew_threshold = 1.0
dist_stats = {}
dist_stats_transformed = {}

for col in key_indicators:
    series = df[col].dropna()
    if series.empty:
        continue

    dist_stats[col] = {
        "mean": series.mean(),
        "median": series.median(),
        "std": series.std(),
        "skewness": series.skew(),
        "kurtosis": series.kurtosis(),
    }

    skew_val = dist_stats[col]["skewness"]
    if skew_val > skew_threshold:
        shift = 0
        if series.min() <= 0:
            shift = abs(series.min()) + 1e-6
        transformed = np.log1p(series + shift)

        dist_stats_transformed[col] = {
            "transformation": f"log1p(+{shift:.6f})" if shift else "log1p",
            "mean": transformed.mean(),
            "median": transformed.median(),
            "std": transformed.std(),
            "skewness": transformed.skew(),
            "kurtosis": transformed.kurtosis(),
        }
    else:
        dist_stats_transformed[col] = {
            "transformation": "none",
            **dist_stats[col],
        }

print("Key indicator statistics (raw vs. skew-corrected):")
for col in dist_stats.keys():
    raw = dist_stats[col]
    adj = dist_stats_transformed[col]
    print(f"\n  {col}")
    print(f"     Raw -> Mean: {raw['mean']:.2f}, Median: {raw['median']:.2f}, Skew: {raw['skewness']:.2f}")
    print(f"     Adj -> {adj['transformation']}, Mean: {adj['mean']:.2f}, Median: {adj['median']:.2f}, Skew: {adj['skewness']:.2f}")

results['eda'] = {
    'missing_analysis': significant_missing.to_dict(),
    'distribution_stats': dist_stats,
    'distribution_stats_adjusted': dist_stats_transformed
}


[3/5] DISTRIBUTION ANALYSIS
Key indicator statistics (raw vs. skew-corrected):

  SDG_NAT_XGDP.FSGOV
     Raw -> Mean: 3.78, Median: 3.39, Skew: 1.23
     Adj -> log1p, Mean: 1.48, Median: 1.48, Skew: -0.06

  Foreign direct investment, net inflows (% of GDP)
     Raw -> Mean: 4.12, Median: 2.32, Skew: 6.18
     Adj -> log1p(+17.292118), Mean: 3.08, Median: 3.03, Skew: 0.22

  Agriculture, forestry, and fishing, value added (% of GDP)
     Raw -> Mean: 18.54, Median: 17.44, Skew: 0.69
     Adj -> none, Mean: 18.54, Median: 17.44, Skew: 0.69

  Industry (including construction), value added (% of GDP)
     Raw -> Mean: 27.04, Median: 23.86, Skew: 1.64
     Adj -> log1p, Mean: 3.23, Median: 3.21, Skew: -0.02

  Services, value added (% of GDP)
     Raw -> Mean: 47.14, Median: 47.08, Skew: 0.17
     Adj -> none, Mean: 47.14, Median: 47.08, Skew: 0.17


In [55]:
# [4/5] Outlier detection
print("\n[4/5] OUTLIER DETECTION")
print("-" * 40)

outlier_details = {}
outlier_adjusted_stats = {}

for col in key_indicators:
    series = df[col].dropna()
    if series.empty:
        continue

    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outlier_mask = (series < lower) | (series > upper)
    outlier_count = int(outlier_mask.sum())
    outlier_pct = (outlier_count / len(series)) * 100 if len(series) else 0.0

    outlier_details[col] = {
        'q1': Q1,
        'q3': Q3,
        'iqr': IQR,
        'lower_bound': lower,
        'upper_bound': upper,
        'outlier_count': outlier_count,
        'outlier_pct': outlier_pct
    }

    if outlier_count > 0:
        winsorized = series.clip(lower, upper)
        outlier_adjusted_stats[col] = {
            'method': 'winsorized_iqr',
            'mean': winsorized.mean(),
            'median': winsorized.median(),
            'std': winsorized.std(),
            'skewness': winsorized.skew(),
            'kurtosis': winsorized.kurtosis()
        }
    else:
        outlier_adjusted_stats[col] = {
            'method': 'none_needed',
            'mean': series.mean(),
            'median': series.median(),
            'std': series.std(),
            'skewness': series.skew(),
            'kurtosis': series.kurtosis()
        }

if outlier_details:
    print("Outliers detected:")
    for col, details in sorted(outlier_details.items(), key=lambda item: item[1]['outlier_count'], reverse=True):
        count = details['outlier_count']
        pct = details['outlier_pct']
        print(f"   - {col[:60]}: {count} outliers ({pct:.1f}%)")
        if count > 0:
            print(f"       Bounds: [{details['lower_bound']:.2f}, {details['upper_bound']:.2f}] after IQR winsorization")
else:
    print("   [OK] No outliers detected within the monitored indicators")

results.setdefault('eda', {})
results['eda']['outlier_summary'] = outlier_details
results['eda']['outlier_adjusted_stats'] = outlier_adjusted_stats


[4/5] OUTLIER DETECTION
----------------------------------------
Outliers detected:
   - Foreign direct investment, net inflows (% of GDP): 118 outliers (8.7%)
       Bounds: [-4.59, 10.47] after IQR winsorization
   - Industry (including construction), value added (% of GDP): 95 outliers (7.0%)
       Bounds: [-1.97, 51.66] after IQR winsorization
   - Services, value added (% of GDP): 56 outliers (4.1%)
       Bounds: [20.87, 72.55] after IQR winsorization
   - SDG_NAT_XGDP.FSGOV: 45 outliers (3.3%)
       Bounds: [-1.47, 8.65] after IQR winsorization
   - Agriculture, forestry, and fishing, value added (% of GDP): 10 outliers (0.7%)
       Bounds: [-20.61, 55.36] after IQR winsorization


In [56]:
print("\n[5/5] CORRELATION PREVIEW")
print("-" * 40)

# Get education and labour indicators
if metadata and 'feature_categories' in metadata:
    edu_cols = [col for col in metadata['feature_categories']['education'] 
                if col in df.columns and df[col].notna().sum() > 10][:3]
    labour_cols = [col for col in metadata['feature_categories']['labour'] 
                    if col in df.columns and df[col].notna().sum() > 10][:3]
else:
    edu_cols = [col for col in df.columns if 'education' in col.lower() or 'literacy' in col.lower()][:3]
    labour_cols = [col for col in df.columns if 'unemployment' in col.lower()][:3]

if edu_cols and labour_cols:
    print(f"Education indicators: {len(edu_cols)}")
    print(f"Labour indicators: {len(labour_cols)}")
    
    corr_preview = {}
    for edu_col in edu_cols:
        for labour_col in labour_cols:
            valid_data = df[[edu_col, labour_col]].dropna()
            if len(valid_data) > 30:
                corr, p_value = pearsonr(valid_data[edu_col], valid_data[labour_col])
                corr_preview[f"{edu_col[:30]} vs {labour_col[:30]}"] = {
                    "correlation": float(np.ravel(corr)[0]),
                    "p_value": float(np.ravel(p_value)[0])
                }
                # corr_preview[f"{edu_col[:30]} vs {labour_col[:30]}"] = {
                #     'correlation': corr,
                #     'p_value': p_value
                # }
    
    if corr_preview:
        print("\nSample correlations:")
        for pair, vals in list(corr_preview.items())[:5]:
            sig = "***" if vals['p_value'] < 0.001 else "**" if vals['p_value'] < 0.01 else "*" if vals['p_value'] < 0.05 else ""
            print(f"   - {pair}: r={vals['correlation']:.3f} {sig}")


[5/5] CORRELATION PREVIEW
----------------------------------------
Education indicators: 3
Labour indicators: 3

Sample correlations:
   - SDG_NAT_AIR.1.GLAST vs Unemployment, youth total (% o: r=0.327 ***
   - SDG_NAT_AIR.1.GLAST vs Unemployment with advanced edu: r=0.032 
   - SDG_NAT_AIR.1.GLAST vs Unemployment, total (% of tota: r=0.302 ***
   - SDG_NAT_AIR.1.GLAST.F vs Unemployment, youth total (% o: r=0.349 ***
   - SDG_NAT_AIR.1.GLAST.F vs Unemployment with advanced edu: r=0.035 


## 4. Correlation and Dependency Analysis

In [57]:
"""Comprehensive correlation and dependency analysis (UPDATED WITH SDG)."""
print("CORRELATION & DEPENDENCY ANALYSIS (WITH SDG INDICATORS)")

# Get feature categories
if metadata and 'feature_categories' in metadata:
    edu_indicators = [col for col in metadata['feature_categories']['education'] 
                        if col in df.columns and df[col].notna().sum() > 50]
    
    labour_indicators = [col for col in metadata['feature_categories']['labour'] 
                        if col in df.columns and df[col].notna().sum() > 50]
    
    economic_indicators = [col for col in metadata['feature_categories']['economic'] 
                            if col in df.columns and df[col].notna().sum() > 50]
else:
    edu_indicators = [col for col in df.columns 
                        if any(kw in col.lower() for kw in ['education', 'literacy', 'school', 'enrollment'])
                        and df[col].notna().sum() > 50]
    labour_indicators = [col for col in df.columns 
                        if 'unemployment' in col.lower() and df[col].notna().sum() > 50]
    economic_indicators = [col for col in df.columns 
                            if any(kw in col.lower() for kw in ['gdp', 'investment']) 
                            and df[col].notna().sum() > 50]

# Include SDG indicators
sdg_indicators = [col for col in sdg_columns if df[col].notna().sum() > 50][:10]

print(f"\n[1/3] FEATURE SELECTION")
print("-" * 40)
print(f"Education indicators: {len(edu_indicators)}")
print(f"Labour indicators: {len(labour_indicators)}")
print(f"Economic indicators: {len(economic_indicators)}")
print(f"SDG indicators (included): {len(sdg_indicators)}")

CORRELATION & DEPENDENCY ANALYSIS (WITH SDG INDICATORS)

[1/3] FEATURE SELECTION
----------------------------------------
Education indicators: 55
Labour indicators: 3
Economic indicators: 8
SDG indicators (included): 10


In [58]:
# Pearson correlation
print(f"\n[2/3] PEARSON CORRELATION ANALYSIS")
print("-" * 40)

correlation_results = []

# Traditional education-labour correlations
for edu_col in edu_indicators[:5]:
    for labour_col in labour_indicators[:3]:
        valid_data = df[[edu_col, labour_col]].dropna()
        
        if len(valid_data) >= 30:
            corr, p_value = pearsonr(valid_data[edu_col], valid_data[labour_col])
            
            correlation_results.append({
                'education_indicator': edu_col,
                'labour_indicator': labour_col,
                'correlation': corr,
                'p_value': p_value,
                'sample_size': len(valid_data),
                'significance': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns',
                'type': 'traditional'
            })

# SDG-labour correlations (NEW)
for sdg_col in sdg_indicators[:5]:
    for labour_col in labour_indicators[:3]:
        valid_data = df[[sdg_col, labour_col]].dropna()
        
        if len(valid_data) >= 30:
            try:
                corr, p_value = pearsonr(valid_data[sdg_col], valid_data[labour_col])
                
                correlation_results.append({
                    'education_indicator': sdg_col,
                    'labour_indicator': labour_col,
                    'correlation': corr,
                    'p_value': p_value,
                    'sample_size': len(valid_data),
                    'significance': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns',
                    'type': 'sdg'
                })
            except:
                pass
# Sort by absolute correlation
correlation_results.sort(key=lambda x: abs(x['correlation']), reverse=True)

print(f"Computed {len(correlation_results)} correlation pairs")
print(f"   Traditional: {sum(1 for r in correlation_results if r['type'] == 'traditional')}")
print(f"   SDG-based: {sum(1 for r in correlation_results if r['type'] == 'sdg')}")

print("\nTop 10 strongest correlations (all types):")
for i, result in enumerate(correlation_results[:10], 1):
    print(f"\n   [{i}] [{result['type'].upper()}] {result['education_indicator'][:40]}")
    print(f"       -> {result['labour_indicator'][:40]}")
    print(f"       r = {result['correlation']:.3f} (p={result['p_value']:.4f}) {result['significance']}")
    print(f"       n = {result['sample_size']}")

results['correlations'] = correlation_results


[2/3] PEARSON CORRELATION ANALYSIS
----------------------------------------
Computed 30 correlation pairs
   Traditional: 15
   SDG-based: 15

Top 10 strongest correlations (all types):

   [1] [TRADITIONAL] SDG_NAT_AIR.1.GLAST.F
       -> Unemployment, youth total (% of total la
       r = 0.349 (p=0.0000) ***
       n = 1350

   [2] [SDG] SDG_NAT_AIR.1.GLAST.F
       -> Unemployment, youth total (% of total la
       r = 0.349 (p=0.0000) ***
       n = 1350

   [3] [TRADITIONAL] SDG_NAT_AIR.1.GLAST.F
       -> Unemployment, total (% of total labor fo
       r = 0.335 (p=0.0000) ***
       n = 1350

   [4] [SDG] SDG_NAT_AIR.1.GLAST.F
       -> Unemployment, total (% of total labor fo
       r = 0.335 (p=0.0000) ***
       n = 1350

   [5] [TRADITIONAL] SDG_NAT_AIR.1.GLAST.GPIA
       -> Unemployment, total (% of total labor fo
       r = 0.328 (p=0.0000) ***
       n = 1350

   [6] [SDG] SDG_NAT_AIR.1.GLAST.GPIA
       -> Unemployment, total (% of total labor fo
       r = 0.328 (p=0

In [59]:
# plotly heatmap for combined education (incl. SDG) vs labour correlations
print("\n[Heatmap] Visualizing education ↔ labour correlations (traditional + SDG)")

corr_source = results.get('correlations', correlation_results)

if not corr_source:
    print("   ⚠️ No correlation results to plot. Run the correlation analysis first.")
    corr_df = pd.DataFrame()
    heatmap_df = pd.DataFrame()
else:
    corr_rows = []
    for entry in corr_source:
        edu_indicator = entry.get('education_indicator', 'Unknown education indicator')
        labour_indicator = entry.get('labour_indicator', 'Unknown labour indicator')
        corr_value = float(entry.get('correlation', np.nan))
        corr_type = entry.get('type', 'traditional')

        corr_rows.append({
            "Education": edu_indicator,
            "Labour": labour_indicator,
            "Correlation": corr_value,
            "DisplayEducation": f"SDG | {edu_indicator}" if corr_type == 'sdg' else edu_indicator,
            "Type": corr_type
        })

    corr_df = pd.DataFrame(corr_rows)

    if corr_df.empty:
        heatmap_df = pd.DataFrame()
        print("   ⚠️ No qualifying indicator pairs (n<30) to visualize.")
    else:
        heatmap_df = (
            corr_df
            .pivot_table(index="DisplayEducation", columns="Labour", values="Correlation", aggfunc='mean')
            .sort_index()
        )
        heatmap_df = heatmap_df[sorted(heatmap_df.columns)]

        print(
            f"   Rendering {len(corr_df)} correlations across "
            f"{heatmap_df.shape[0]} education vs {heatmap_df.shape[1]} labour indicators"
        )

        fig = go.Figure(
            data=go.Heatmap(
                z=heatmap_df.values,
                x=heatmap_df.columns,
                y=heatmap_df.index,
                colorscale="RdBu",
                zmin=-1,
                zmax=1,
                colorbar=dict(title="Pearson r"),
                hovertemplate="Labour: %{x}<br>Education: %{y}<br>r = %{z:.3f}<extra></extra>"
            )
        )

        fig.update_layout(
            title="Education (incl. SDG) vs Labour Correlations",
            xaxis_title="Labour Indicators",
            yaxis_title="Education / SDG Indicators",
            height=800,
            margin=dict(l=220, r=120, t=80, b=220)
        )

        fig.show()


[Heatmap] Visualizing education ↔ labour correlations (traditional + SDG)
   Rendering 30 correlations across 10 education vs 3 labour indicators


In [60]:
# [3/3] Multicollinearity check (VIF)
print("\n[3/3] MULTICOLLINEARITY CHECK (VIF)")

# Select subset of indicators for VIF
def recommend_vif_treatment(dataframe, max_vif=10.0, max_removals=5):
    """Run iterative VIF on a copy and return retained scores plus recommended drops."""
    working_df = dataframe.copy()
    dropped = []
    final_scores = []

    while working_df.shape[1] >= 2:
        scaler = StandardScaler()
        scaled = scaler.fit_transform(working_df)
        vif_scores = [
            {'indicator': col, 'vif': float(variance_inflation_factor(scaled, i))}
            for i, col in enumerate(working_df.columns)
        ]
        # Keep latest scores for reporting
        final_scores = vif_scores

        worst = max(vif_scores, key=lambda x: x['vif'])
        if worst['vif'] <= max_vif or len(dropped) >= max_removals:
            break

        dropped.append(worst)
        working_df = working_df.drop(columns=[worst['indicator']])

    return final_scores, dropped

vif_indicators = edu_indicators[:5] + labour_indicators[:3]
vif_data = df[vif_indicators].dropna()

if len(vif_data) > 10:
    try:
        final_vif_scores, recommended_drops = recommend_vif_treatment(vif_data, max_vif=10.0)

        if recommended_drops:
            print("Recommended indicator removals to reduce multicollinearity (evaluated on a copy):")
            for entry in recommended_drops:
                print(f"   - {entry['indicator'][:60]}: VIF {entry['vif']:.2f}")
        else:
            print("   No removals recommended; all monitored indicators meet the VIF threshold")

        if final_vif_scores:
            print("Remaining VIF scores (>10 indicates high multicollinearity):")
            for entry in final_vif_scores:
                status = "HIGH" if entry['vif'] > 10 else "OK"
                print(f"   - {entry['indicator'][:60]}: {entry['vif']:.2f} {status}")
        else:
            print("   WARNING: Not enough indicators remain for VIF calculation after evaluation")

        results['vif'] = final_vif_scores
        results['vif_recommendations'] = recommended_drops
    except Exception as e:
        print(f"   WARNING: VIF calculation error: {str(e)}")
else:
    print("   WARNING: Not enough complete rows for VIF analysis")


[3/3] MULTICOLLINEARITY CHECK (VIF)
Recommended indicator removals to reduce multicollinearity (evaluated on a copy):
   - SDG_NAT_AIR.1.GLAST: VIF 3078.69
   - SDG_NAT_AIR.1.GLAST.F: VIF 77.93
   - Unemployment, total (% of total labor force): VIF 12.76
Remaining VIF scores (>10 indicates high multicollinearity):
   - SDG_NAT_AIR.1.GLAST.GPIA: 1.35 OK
   - SDG_NAT_AIR.1.GLAST.M: 3.80 OK
   - SDG_NAT_AIR.2.GPV.GLAST: 4.04 OK
   - Unemployment, youth total (% of total labor force ages 15-24: 1.26 OK
   - Unemployment with advanced education (% of total labor force: 1.09 OK


## 5. Statistical Relationship Models (ML Models)

In [61]:
"""Build statistical models to quantify education-labour relationships with expanded features and tuned models."""
print("\n" + "=" * 80)
print("STATISTICAL RELATIONSHIP MODELS (ENHANCED)")
print("=" * 80)

# Base dataframe already prepared earlier
model_base_df = df.copy()

# Define keyword buckets for candidate predictors
education_keywords = ['education', 'school', 'literacy', 'enrollment', 'completion']
economic_keywords = ['gdp', 'value added', 'services', 'industry', 'agriculture', 'investment']
demographic_keywords = ['population', 'age']

candidate_features = [
    col for col in model_base_df.columns
    if any(kw in col.lower() for kw in education_keywords + economic_keywords + demographic_keywords)
]

# Labour targets
if metadata and 'feature_categories' in metadata:
    labour_indicators = [
        col for col in metadata['feature_categories']['labour']
        if col in model_base_df.columns and model_base_df[col].notna().sum() > 150
    ]
else:
    labour_indicators = [
        col for col in model_base_df.columns
        if 'unemployment' in col.lower() and model_base_df[col].notna().sum() > 150
    ]

if not labour_indicators:
    labour_indicators = [
        col for col in model_base_df.columns
        if 'unemployment' in col.lower() and model_base_df[col].notna().sum() > 80
    ]

print(f"Candidate predictor pool: {len(candidate_features)} columns")
print(f"Labour targets: {len(labour_indicators)}")


def select_top_features(target_col, top_n=12):
    corr_pairs = []
    for col in candidate_features:
        if col == target_col or col not in model_base_df.columns:
            continue
        valid = model_base_df[[col, target_col]].dropna()
        if len(valid) < 60:
            continue
        corr_val = valid[col].corr(valid[target_col])
        if pd.notna(corr_val):
            corr_pairs.append((col, abs(corr_val)))
    corr_pairs.sort(key=lambda x: x[1], reverse=True)
    selected = [col for col, _ in corr_pairs[:top_n]]
    return selected


def add_feature_engineering(X):
    literacy_cols = [c for c in X.columns if 'literacy' in c.lower()]
    if len(literacy_cols) >= 2:
        X['literacy_composite'] = X[literacy_cols].mean(axis=1)
    services_col = next((c for c in X.columns if 'services, value added (% of gdp)' in c.lower()), None)
    if services_col:
        X['services_dominance'] = X[services_col] / 100.0
    return X


model_results = []


def run_models_for_target(labour_target):
    selected_features = select_top_features(labour_target, top_n=12)
    if not selected_features:
        print(f"   WARNING: No correlated predictors found for {labour_target}")
        return

    feature_df = model_base_df[selected_features + [labour_target]].dropna()
    if len(feature_df) < 200:
        print(f"   WARNING: Insufficient rows ({len(feature_df)}) for {labour_target}")
        return

    X = feature_df.drop(columns=[labour_target]).copy()
    X = add_feature_engineering(X)
    X = X.fillna(X.median())
    y = feature_df[labour_target].copy()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    models = {
        'Linear Regression': LinearRegression(),
        'Ridge': Ridge(alpha=1.0),
        'Lasso': Lasso(alpha=0.05),
        'Random Forest (tuned)': RandomForestRegressor(
            n_estimators=200,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=4,
            random_state=42
        ),
        'Gradient Boosting (tuned)': GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=5,
            min_samples_split=10,
            subsample=0.8,
            random_state=42
        )
    }

    linear_models = {'Linear Regression', 'Ridge', 'Lasso'}

    print(f"\nTraining models for target: {labour_target}")
    print(f"   Predictors used: {len(X.columns)}")

    for name, model in models.items():
        try:
            if name in linear_models:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            r2 = r2_score(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)

            result_entry = {
                'target': labour_target,
                'model': name,
                'r2': r2,
                'rmse': rmse,
                'mae': mae,
                'train_size': len(X_train),
                'test_size': len(X_test),
                'feature_count': len(X.columns),
                'features': list(X.columns)
            }

            if name == 'Gradient Boosting (tuned)':
                cv_scores = cross_val_score(
                    model, X, y, scoring='r2', cv=5
                )
                result_entry['cv_mean_r2'] = float(cv_scores.mean())
                result_entry['cv_std_r2'] = float(cv_scores.std())

            model_results.append(result_entry)

            print(f"   {name}: R2={r2:.3f} | RMSE={rmse:.2f} | MAE={mae:.2f}")
            if 'cv_mean_r2' in result_entry:
                print(f"      CV R2 mean={result_entry['cv_mean_r2']:.3f} (std={result_entry['cv_std_r2']:.3f})")
        except Exception as exc:
            print(f"   WARNING: {name} failed for {labour_target}: {exc}")


for labour_target in labour_indicators:
    print("\n" + "=" * 60)
    print(f"TARGET: {labour_target}")
    print("=" * 60)
    run_models_for_target(labour_target)

results['ml_models'] = model_results


STATISTICAL RELATIONSHIP MODELS (ENHANCED)
Candidate predictor pool: 19 columns
Labour targets: 3

TARGET: Unemployment, youth total (% of total labor force ages 15-24)

Training models for target: Unemployment, youth total (% of total labor force ages 15-24)
   Predictors used: 14
   Linear Regression: R2=0.543 | RMSE=11.96 | MAE=8.27
   Ridge: R2=0.543 | RMSE=11.96 | MAE=8.30
   Lasso: R2=0.541 | RMSE=11.98 | MAE=8.40
   Random Forest (tuned): R2=0.934 | RMSE=4.53 | MAE=2.51
   Gradient Boosting (tuned): R2=0.960 | RMSE=3.52 | MAE=2.20
      CV R2 mean=0.180 (std=0.462)

TARGET: Unemployment with advanced education (% of total labor force with advanced education)

Training models for target: Unemployment with advanced education (% of total labor force with advanced education)
   Predictors used: 13
   Linear Regression: R2=0.109 | RMSE=6.09 | MAE=4.05
   Ridge: R2=0.110 | RMSE=6.09 | MAE=4.05
   Lasso: R2=0.114 | RMSE=6.07 | MAE=4.04
   Random Forest (tuned): R2=0.636 | RMSE=3.89 | 

In [62]:
# Summary
print("MODEL PERFORMANCE SUMMARY")

if model_results:
    best_models = sorted(model_results, key=lambda x: x['r2'], reverse=True)[:5]
    print("\nTop 5 models by R2:")
    for i, result in enumerate(best_models, 1):
        sdg_note = " (with SDG)" if result.get('includes_sdg') else ""
        print(f"\n   [{i}] {result['model']} -> {result['target'][:40]}{sdg_note}")
        print(f"       R2 = {result['r2']:.3f}, RMSE = {result['rmse']:.3f}")

MODEL PERFORMANCE SUMMARY

Top 5 models by R2:

   [1] Gradient Boosting (tuned) -> Unemployment, total (% of total labor fo
       R2 = 0.996, RMSE = 0.493

   [2] Random Forest (tuned) -> Unemployment, total (% of total labor fo
       R2 = 0.987, RMSE = 0.850

   [3] Gradient Boosting (tuned) -> Unemployment, youth total (% of total la
       R2 = 0.960, RMSE = 3.516

   [4] Random Forest (tuned) -> Unemployment, youth total (% of total la
       R2 = 0.934, RMSE = 4.533

   [5] Ridge -> Unemployment, total (% of total labor fo
       R2 = 0.930, RMSE = 1.995


## 6. Time-Series Forecasting

In [63]:
print("TIME-SERIES FORECASTING")

forecast_targets = [
    col for col in df.columns
    if 'unemployment' in col.lower() and df[col].notna().sum() > 100
][:2]

if not forecast_targets:
    print("WARNING: No suitable indicators for forecasting")
else:
    alias_map = {col: f"{col}_target_ts" for col in forecast_targets}
    forecast_df = df.rename(columns=alias_map)

    eda_results = results.get('eda', {})
    dist_stats = eda_results.get('distribution_stats', {})
    outlier_summary = eda_results.get('outlier_summary', {})
    log_candidates = {
        col for col, stats in dist_stats.items()
        if abs(stats.get('skewness', 0)) > 1.0
    }

    def prepare_ts_series(target_col, apply_treatment=False):
        alias_col = alias_map.get(target_col, target_col)
        ts = (
            forecast_df.groupby('year')[alias_col]
            .mean()
            .reset_index()
            .rename(columns={alias_col: 'value'})
            .dropna()
        )
        treatment_report = {}

        if apply_treatment and not ts.empty:
            series = ts['value'].copy()
            if target_col in log_candidates:
                shift = 0.0
                if (series <= 0).any():
                    shift = abs(series.min()) + 1e-6
                series = np.log1p(series + shift)
                treatment_report['log_transform'] = {'shift': shift}

            bounds = outlier_summary.get(target_col)
            if bounds:
                lower = bounds.get('lower_bound')
                upper = bounds.get('upper_bound')
                series = series.clip(lower=lower, upper=upper)
                treatment_report['winsorization'] = {'lower': lower, 'upper': upper}

            ts['value'] = series

        return ts, treatment_report

    def run_forecast(ts_df, view_label, treatment_report):
        if len(ts_df) < 10:
            print(f"   WARNING: View '{view_label}' has insufficient time points ({len(ts_df)})")
            return None

        years = ts_df['year'].values
        values = ts_df['value'].values

        z = np.polyfit(years, values, 1)
        p = np.poly1d(z)
        fitted = p(years)
        residuals = values - fitted
        rmse_hist = float(np.sqrt(np.mean(residuals ** 2)))
        try:
            r2_hist = float(r2_score(values, fitted))
        except Exception:
            r2_hist = float('nan')

        trend_direction = "increasing" if z[0] > 0 else "decreasing"

        future_years = np.arange(int(years[-1]) + 1, int(years[-1]) + 4)
        forecast_values = p(future_years)

        print(f"\nView '{view_label}':")
        print(f"   Time series length: {len(ts_df)} years")
        print(f"   Range: {ts_df['year'].min()} - {ts_df['year'].max()}")
        print(f"   Trend: {trend_direction} ({z[0]:.4f} per year)")
        print(f"   Historical RMSE vs. trend: {rmse_hist:.2f}, R2: {r2_hist:.3f}")
        if treatment_report:
            print(f"   Treatments applied: {treatment_report}")

        print("\n   Forecasts:")
        view_forecasts = []
        for year, value in zip(future_years, forecast_values):
            print(f"      {int(year)}: {value:.2f}")
            view_forecasts.append({'year': int(year), 'value': float(value)})

        return {
            'view': view_label,
            'treatment_report': treatment_report,
            'trend_coefficient': float(z[0]),
            'trend_intercept': float(z[1]),
            'trend_direction': trend_direction,
            'historical_rmse': rmse_hist,
            'historical_r2': r2_hist,
            'historical_points': ts_df.to_dict('records'),
            'forecasts': view_forecasts
        }

    print(f"\nForecasting targets: {len(forecast_targets)}")

    forecast_results = []

    for target in forecast_targets:
        print(f"\nFORECASTING: {target}")
        target_views = []

        ts_raw, raw_report = prepare_ts_series(target, apply_treatment=False)
        raw_result = run_forecast(ts_raw, 'raw', raw_report)
        if raw_result:
            target_views.append(raw_result)

        ts_treated, treated_report = prepare_ts_series(target, apply_treatment=True)
        treated_result = run_forecast(ts_treated, 'treated', treated_report)
        if treated_result:
            target_views.append(treated_result)

        forecast_results.append({
            'target': target,
            'views': target_views
        })

    results['forecasts'] = forecast_results

TIME-SERIES FORECASTING

Forecasting targets: 2

FORECASTING: Unemployment, youth total (% of total labor force ages 15-24)

View 'raw':
   Time series length: 25 years
   Range: 2000 - 2024
   Trend: increasing (0.0013 per year)
   Historical RMSE vs. trend: 0.47, R2: 0.000

   Forecasts:
      2025: 17.58
      2026: 17.58
      2027: 17.58

View 'treated':
   Time series length: 25 years
   Range: 2000 - 2024
   Trend: increasing (0.0013 per year)
   Historical RMSE vs. trend: 0.47, R2: 0.000

   Forecasts:
      2025: 17.58
      2026: 17.58
      2027: 17.58

FORECASTING: Unemployment with advanced education (% of total labor force with advanced education)

View 'raw':
   Time series length: 25 years
   Range: 2000 - 2024
   Trend: increasing (0.0119 per year)
   Historical RMSE vs. trend: 0.39, R2: 0.046

   Forecasts:
      2025: 11.69
      2026: 11.70
      2027: 11.71

View 'treated':
   Time series length: 25 years
   Range: 2000 - 2024
   Trend: increasing (0.0119 per year)

In [64]:
print("SAVING RESULTS")
output_dir = data_dir.parent / 'results'
output_dir.mkdir(exist_ok=True)

from numbers import Number

def make_serializable(obj):
    if isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    elif pd.isna(obj):
        return None
    elif isinstance(obj, (str, int, float, bool, type(None))):
        return obj
    else:
        return str(obj)


def format_float(val, precision=4, default='N/A'):
    if isinstance(val, Number) and not pd.isna(val):
        return f"{val:.{precision}f}"
    return default


results_clean = make_serializable(results)

with open(output_dir / 'analysis_results.json', 'w') as f:
    json.dump(results_clean, f, indent=2)

print(f"[OK] Saved: analysis_results.json")
print(f"   Location: {output_dir}")

# Save summary report
with open(output_dir / 'analysis_summary.txt', 'w', encoding='utf-8') as f:
    f.write("=" * 80 + "\n")
    f.write("EDUCATION-LABOUR ANALYTICS SUMMARY (WITH SDG DATA)\n")
    f.write("=" * 80 + "\n\n")
    
    # SDG Analysis
    if 'sdg_analysis' in results:
        f.write("0. SDG INDICATOR ANALYSIS\n")
        f.write("-" * 40 + "\n")
        sdg_data = results['sdg_analysis']
        f.write(f"Total SDG indicators: {len(sdg_data.get('availability', {}))}\n")
        
        if sdg_data.get('sdg_labour_correlations'):
            f.write(f"\nTop 3 SDG-Labour correlations:\n")
            for i, corr in enumerate(sdg_data['sdg_labour_correlations'][:3], 1):
                f.write(f"\n   [{i}] {corr['sdg_indicator']}\n")
                f.write(f"       -> {corr['labour_indicator']}\n")
                f.write(f"       r = {corr['correlation']:.3f} (p={corr['p_value']:.4f})\n")
        f.write("\n\n")
    
    f.write("1. CORRELATION ANALYSIS\n")
    f.write("-" * 40 + "\n")
    if 'correlations' in results:
        f.write(f"Total correlation pairs analyzed: {len(results['correlations'])}\n")
        
        traditional = [c for c in results['correlations'] if c.get('type') == 'traditional']
        sdg_based = [c for c in results['correlations'] if c.get('type') == 'sdg']
        f.write(f"   Traditional: {len(traditional)}\n")
        f.write(f"   SDG-based: {len(sdg_based)}\n")
        
        f.write(f"\nTop 5 strongest correlations:\n")
        for i, corr in enumerate(results['correlations'][:5], 1):
            f.write(f"\n   [{i}] [{corr.get('type', 'N/A').upper()}] {corr['education_indicator']}\n")
            f.write(f"       -> {corr['labour_indicator']}\n")
            f.write(f"       r = {corr['correlation']:.3f} (p={corr['p_value']:.4f})\n")
    
    f.write("\n\n2. ML MODEL PERFORMANCE\n")
    f.write("-" * 40 + "\n")
    if 'ml_models' in results:
        best_models = sorted(results['ml_models'], key=lambda x: x['r2'], reverse=True)[:5]
        f.write(f"\nTop 5 models by R2:\n")
        for i, model in enumerate(best_models, 1):
            sdg_note = " (with SDG features)" if model.get('includes_sdg') else ""
            f.write(f"\n   [{i}] {model['model']}{sdg_note}\n")
            f.write(f"       Target: {model['target']}\n")
            f.write(f"       R2 = {model['r2']:.3f}\n")
            f.write(f"       RMSE = {model['rmse']:.3f}\n")
    
    f.write("\n\n3. TIME-SERIES FORECASTS\n")
    f.write("-" * 40 + "\n")
    if 'forecasts' in results and results['forecasts']:
        for forecast in results['forecasts']:
            target_name = forecast.get('target', 'Unknown target')
            f.write(f"\n   {target_name}\n")
            views = forecast.get('views')
            if views:
                for view in views:
                    view_label = view.get('view', 'view')
                    trend_dir = view.get('trend_direction', 'N/A')
                    trend_coef = format_float(view.get('trend_coefficient'), precision=4)
                    hist_r2 = format_float(view.get('historical_r2'), precision=3)
                    hist_rmse = format_float(view.get('historical_rmse'), precision=2)
                    f.write(f"   [{view_label}] Trend: {trend_dir} ({trend_coef}/year)\n")
                    f.write(f"      Historical RMSE: {hist_rmse}, R2: {hist_r2}\n")
                    if view.get('treatment_report'):
                        f.write(f"      Treatments: {view['treatment_report']}\n")
                    forecasts_list = view.get('forecasts', [])
                    if forecasts_list:
                        f.write("      Forecasts:\n")
                        for pred in forecasts_list:
                            year = pred.get('year', 'N/A')
                            value = format_float(pred.get('value'), precision=2)
                            f.write(f"         {year}: {value}\n")
                    else:
                        f.write("      Forecasts: N/A\n")
            else:
                trend_dir = forecast.get('trend_direction', 'N/A')
                trend_coef = format_float(forecast.get('trend_coefficient'), precision=4)
                f.write(f"   Trend: {trend_dir} ({trend_coef}/year)\n")
                forecasts_list = forecast.get('forecasts', [])
                if forecasts_list:
                    f.write("   Forecasts:\n")
                    for pred in forecasts_list:
                        year = pred.get('year', 'N/A')
                        value = format_float(pred.get('value'), precision=2)
                        f.write(f"      {year}: {value}\n")
                else:
                    f.write("   Forecasts: N/A\n")
    else:
        f.write("\n   No time-series forecasts available.\n")

print(f"[OK] Saved: analysis_summary.txt")
print("\n" + "=" * 80)
print("ANALYSIS PIPELINE COMPLETE (WITH SDG INTEGRATION)")
print("=" * 80)

SAVING RESULTS
[OK] Saved: analysis_results.json
   Location: results
[OK] Saved: analysis_summary.txt

ANALYSIS PIPELINE COMPLETE (WITH SDG INTEGRATION)
